# Generate async request JSON file for ChatComplete API: Item Profile 

In [1]:
import pandas as pd
from tqdm.notebook import tqdm
import json
import re
from collections import OrderedDict
import numpy as np
import pickle

In [2]:
import openai
import os
with open("../specs/.openai_api_key", "r") as f:
    ss = f.readline().strip()

os.environ["OPENAI_API_KEY"] = ss

In [3]:
from openai import OpenAI
client = OpenAI()

In [4]:
from openai import OpenAI
client = OpenAI()

model_list = client.models.list()

In [5]:
for x in model_list: print(x)

Model(id='dall-e-3', created=1698785189, object='model', owned_by='system')
Model(id='gpt-4-1106-preview', created=1698957206, object='model', owned_by='system')
Model(id='dall-e-2', created=1698798177, object='model', owned_by='system')
Model(id='tts-1', created=1681940951, object='model', owned_by='openai-internal')
Model(id='tts-1-hd-1106', created=1699053533, object='model', owned_by='system')
Model(id='tts-1-1106', created=1699053241, object='model', owned_by='system')
Model(id='tts-1-hd', created=1699046015, object='model', owned_by='system')
Model(id='text-embedding-3-large', created=1705953180, object='model', owned_by='system')
Model(id='babbage-002', created=1692634615, object='model', owned_by='system')
Model(id='gpt-4-turbo-preview', created=1706037777, object='model', owned_by='system')
Model(id='gpt-4-0125-preview', created=1706037612, object='model', owned_by='system')
Model(id='gpt-4o-mini', created=1721172741, object='model', owned_by='system')
Model(id='gpt-4o-mini-20

In [6]:
# engine_id='gpt-3.5-turbo-16k'
engine_id='gpt-4o-mini'

In [ ]:
def make_json_body(request_id, system_prompt, user_prompt):
    dic = { 
        'model':engine_id,
        'messages':[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        'temperature':0.0,
        'metadata':{'request_id':request_id},
    }
    return json.dumps(dic, ensure_ascii=False)

## The prompt temmplate for generating Item Profile 

In [ ]:
system_prompt = """
You will serve as an assistant to help me summarize which types of users would enjoy a specific product.
I will provide you with the title and a description of the product.
Here are the instructions:
1. I will provide you with information in the form of a JSON string that describes the product:
{
    "title": "the title of the product in Korean", (if there is no title, I will set this value to "None")
    "description": "a description of the product in Korean", (if there is no description, I will set this value to "None")
}

Requirements:
1. Please provide your answer in JSON format, following this structure:
{
    "summarization": "A summarization of what types of users would enjoy this product in Korean" (if you are unable to summarize it, please set this value to "None")
    "reasoning": "briefly explain your reasoning for the summarization in Korean"
}
2. Please ensure that the "summarization" is no longer than 200 words.
3. Please ensure that the "reasoning" is no longer than 200 words.
4. Do not provide any other text outside the JSON string.
"""

In [ ]:
item_df = pd.read_pickle('/Jupyter/dev_src/RLMRec/data/katchers/intermediate/item_df.pkl')

In [ ]:
api_request_file_path = '/Jupyter/dev_src/RLMRec/data/katchers/intermediate/openai-request-items.json'

In [ ]:
with open(api_request_file_path, 'w') as f:
    cnt = 0
    
    for _, row in item_df.iterrows():
        jsonl = make_json_body(
            request_id=f"{row.name}",
            system_prompt=system_prompt,
            user_prompt=str({"title":row['title'], "description":"None"})
            )      
        f.write(jsonl + '\n')

## Analysis generated responses

In [7]:
response_file_path = '/Jupyter/dev_src/RLMRec/data/katchers/intermediate/openai-response-items.json'

In [8]:
in_fname =  response_file_path
datas = []
request_ids = []
# results = {}
with open(in_fname, 'r') as f:
    for i, line in enumerate(f):
        data = json.loads(line)
        if isinstance(data[1] , dict):
            # results[i] = data
            datas.append(data[1]['choices'][0]['message']['content'])
            request_ids.append(data[2]['request_id'])

In [9]:
df = pd.DataFrame({'request_id': request_ids, 'response': datas})
df['response'] = df['response'].str.strip()
df['iid'] = df.request_id.astype(int)
profile_df = df.sort_values(by='iid')

# Generate aynsc request JSON file for Embedding API

In [10]:
profile_df

,request_id,response,iid
33,0,"{\n ""summarization"": ""이 제품은 피부 관리에 관심이 많은 사...",0
0,1,"{\n ""summarization"": ""욕실 청소를 자주 하는 사용자, 청소 ...",1
35,2,"{\n ""summarization"": ""이 제품은 주방에서 음식을 보관하고자 ...",2
81,3,"{\n ""summarization"": ""음식 안전과 청결을 중시하는 가정주부 ...",3
51,4,"{\n ""summarization"": ""일반일회용장갑을 필요로 하는 사용자, ...",4
...,...,...,...
1663,1664,"{\n ""summarization"": ""피부 진정과 보습을 원하는 사용자들이 ...",1664
1667,1665,"{\n ""summarization"": ""이 제품은 다이어트와 건강에 관심이 있...",1665
1665,1666,"{\n ""summarization"": ""주름 개선과 피부 관리를 원하는 사용자...",1666
1664,1667,"{\n ""summarization"": ""안티에이징 효과를 원하는 사용자, 특히...",1667


In [ ]:
engine_id='text-embedding-3-small'

In [ ]:
def make_json_body(engine_id, idx, body):
    dic = { 
        'model':engine_id,
        'input':body,
        'metadata': {"request_id": idx}
    }
    return json.dumps(dic, ensure_ascii=False)

In [ ]:
api_request_file_path = '/Jupyter/dev_src/RLMRec/data/katchers/intermediate/openai-request-items-embed.json'

In [ ]:
with open(api_request_file_path, 'w') as f:
    cnt = 0
    
    for _, row in profile_df.iterrows():
        jsonl = make_json_body(
            engine_id=engine_id,
            idx = row['iid'],
            body=row['response']
            )      
        f.write(jsonl + '\n')

## Analysis generated responses

(1533, 1536)

In [11]:
generated_data_path = '/Jupyter/dev_src/RLMRec/data/katchers/intermediate/openai-response-items-embed.json'

In [12]:
in_fname = generated_data_path
datas = []
dic = OrderedDict()
with open(in_fname, 'r') as f:
    for i, line in enumerate(f):
        data = json.loads(line)
        
        IDX = data[2]['request_id']
        embedding = data[1]['data'][0]['embedding']
        
        dic[IDX] = embedding
result_df = pd.DataFrame.from_records([{'IDX':k, 'embedding':v} for k, v in dic.items()]).sort_values('IDX')
result_df.set_index('IDX', inplace=True)

# Write Item Embedding file

In [13]:
embedding_matrix = np.vstack(result_df['embedding'].values)

In [14]:
with open('/Jupyter/dev_src/RLMRec/data/katchers/itm_emb_np.pkl', 'wb') as file:
    pickle.dump(embedding_matrix, file)

In [15]:
embedding_matrix.shape

(1669, 1536)